## Handling datasets: looking at Gaia data with pandas and spark

Previously, we looked at data from real stars from a .npy file to plot it, and also at how to do calculations over datasets. Here, we'll combine these: we'll read in data from a .csv file (which are more commonly used for real datasets, but also more complicated), do some computation over the data, and plot the result. Our goal here will be to make a "color-magnitude diagram", which is similar to the Hertzsprung-Russell diagrams we already made.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

CSV stands for "comma-separated values"; to get a sense for what this looks like, open up the file `praesape.csv` directly. On Camber, right-click and select "Open With -> Editor".

`pandas` is a Python library for handling heterogeneous data: cases where you've got multiple types, labels, missing entries, etc. It processes these data into a pandas-specific object called a _dataframe_. Many astronomical datasets and simulation results can be interpreted in terms of dataframes. There's many more specific ways of managing data and computations using dataframes - my goal for today is just to make you aware of them as a potential tool!

`pandas` isn't the only dataframe library - later today we'll use Apache Spark to work with the whole Gaia catalog, which will have its own dataframe object.

In [24]:
df = pd.read_csv("pleiades.csv")
# this is a nice example; `read_csv` will often not work out of the box
# and you'll need to change the keyword arguments to adjust 
# (look at the docstring to see what these could be)
df

,source_id,ra,dec,parallax,parallax_error,pmra,pmdec,phot_g_mean_mag,bp_rp
0,67282419337270912,58.491370,25.498417,0.094849,0.371451,0.835777,-4.955170,19.580868,1.317089
1,63774423190205184,58.313006,21.668941,-0.662365,1.006775,-0.218220,-0.834387,20.479963,1.223238
2,63774423190208768,58.317327,21.676712,1.133868,0.397403,7.832908,-12.626946,19.360348,2.558887
3,64161038966959744,57.583515,22.944045,-0.045852,0.216397,5.350917,-6.309565,19.007599,0.937456
4,64161043264453120,57.592234,22.939421,0.367243,0.326565,-0.041847,-1.325775,19.583479,0.973051
...,...,...,...,...,...,...,...,...,...
252679,70510924776259584,58.284305,26.697337,0.294407,0.436231,0.497012,-1.604432,19.829073,1.022099
252680,70510924778519040,58.289924,26.694471,-0.624384,0.628040,1.381304,0.330631,20.303984,0.813198
252681,70510929071679616,58.279979,26.698747,-0.675687,0.632664,-1.707503,-1.458575,20.216742,1.138424
252682,70513879713761280,58.263945,26.706857,-0.480591,0.482409,3.956724,1.566958,18.657028,1.748831


We can access the columns by name:

In [25]:
df.parallax

0         0.094849
1        -0.662365
2         1.133868
3        -0.045852
4         0.367243
            ...   
252679    0.294407
252680   -0.624384
252681   -0.675687
252682   -0.480591
252683    0.401748
Name: parallax, Length: 252684, dtype: float64

And for the most part we can treat them like we do arrays:

In [26]:
df.parallax * 2

0         0.189697
1        -1.324729
2         2.267736
3        -0.091703
4         0.734487
            ...   
252679    0.588813
252680   -1.248768
252681   -1.351375
252682   -0.961183
252683    0.803496
Name: parallax, Length: 252684, dtype: float64

Or if we need to, we can convert them to arrays:

In [27]:
parallax_arr = np.array(df.parallax)
parallax_arr

array([ 0.09484854, -0.66236454,  1.13386814, ..., -0.67568728,
       -0.48059136,  0.40174799])

As with numpy arrays, we can mask out data, but here we can do it across the whole dataframe: for example, here's the dataframe cut to only the entries with positive parallax.

In [28]:
df[df.parallax > 0]

,source_id,ra,dec,parallax,parallax_error,pmra,pmdec,phot_g_mean_mag,bp_rp
0,67282419337270912,58.491370,25.498417,0.094849,0.371451,0.835777,-4.955170,19.580868,1.317089
2,63774423190208768,58.317327,21.676712,1.133868,0.397403,7.832908,-12.626946,19.360348,2.558887
4,64161043264453120,57.592234,22.939421,0.367243,0.326565,-0.041847,-1.325775,19.583479,0.973051
5,66521037596508928,57.092457,23.953635,0.169415,0.685547,0.867109,-0.344517,19.956541,1.122597
7,65385280742642688,58.965397,22.696982,1.523014,0.626083,-11.580336,-19.365866,18.350922,2.572901
...,...,...,...,...,...,...,...,...,...
252676,70510654196114816,58.300002,26.670782,0.303421,0.078987,-2.762817,-2.167052,17.180061,0.999315
252677,70510856056778496,58.295457,26.675040,0.163367,0.191047,-0.858450,-2.545147,18.552332,1.219523
252678,70510890416518144,58.303644,26.684475,0.032127,0.292985,0.667064,-0.011082,19.319767,1.125587
252679,70510924776259584,58.284305,26.697337,0.294407,0.436231,0.497012,-1.604432,19.829073,1.022099


### Exercise

Make a scatter plot of the spatial distribution (defined by RA and Dec) of the stars in `df`, where the opacity of every data point corresponds to its parallax (it's possible to plot a star even if it doesn't have a well-defined parallax). It may be useful to look at the `alpha` parameter of `plt.scatter`, and the `.max()` and `.min()` functions on dataframes.

It's likely that dataframes have taken care of some important steps for you without you seeing it directly: to see what these might be, try asking for the maximum parallax in the dataframe, and then do the same in numpy using `parallax_arr`.

### Exercise

Make a histogram (Google around for the appropriate function) of the parallaxes of the stars in this dataframe. Make sure you see some spread, i.e. you shouldn't have all/almost all of the points in one bar.

You should see a small peak off to the right: try and cut the dataframe down so that it just has these entries. What do you notice about the RA and Dec distribution around this peak? Compute the distance to these stars, and see if it matches the known distance to the Pleiades.

### Exercise

Navigate to the pandas cheat sheet at https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf. We'll rerun the numpy functions activity: pick any function on here (I'll write who's got what on the board, so we don't have clashes), take a few minutes to read the docs and play around with it, and then tell the room what you learned!

From here, we'll run through the Camber tutorial at `gaia-all-sky-map.ipynb` - learning how to work with Spark is out of the scope of the bootcamp, but it'll be good to know it's something we can do!

Before leaving this notebook, run `df.count()` here, and do the same there. Note the difference in runtime and scale of data.